# Imports

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
import operator
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import seaborn as sns
from sklearn import neighbors

# Visualizar Dataset

In [61]:
dataset = pd.read_csv('DataSet\StudentPerformanceFactors.csv')
dataset = dataset.dropna()
# Convert 'Yes' to 1 and 'No' to 0 for all columns in the dataset
dataset = dataset.replace({'Yes': 1, 'No': 0})
dataset.head()


C:\Users\54911\AppData\Local\Temp\ipykernel_19844\2321851433.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset = dataset.replace({'Yes': 1, 'No': 0})


,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,0,7,73,Low,1,0,Low,Medium,Public,Positive,3,0,High School,Near,Male,67
1,19,64,Low,Medium,0,8,59,Low,1,2,Medium,Medium,Public,Negative,4,0,College,Moderate,Female,61
2,24,98,Medium,Medium,1,7,91,Medium,1,2,Medium,Medium,Public,Neutral,4,0,Postgraduate,Near,Male,74
3,29,89,Low,Medium,1,8,98,Medium,1,1,Medium,Medium,Public,Negative,4,0,High School,Moderate,Male,71
4,19,92,Medium,Medium,1,6,65,Medium,1,3,Medium,High,Public,Neutral,4,0,College,Near,Female,70


# Create dummies

In [62]:
# Create dummy variables for categorical columns
categorical_columns = ['Parental_Involvement', 'Access_to_Resources', 'Motivation_Level', 'Family_Income', 'Teacher_Quality', 'Peer_Influence', 'School_Type', 'Parental_Education_Level', 'Distance_from_Home', 'Gender']
dataset = pd.get_dummies(dataset, columns=categorical_columns)
dataset = dataset.replace({True: 1, False: 0})
print(dataset.columns)
dataset.head()

Index(['Hours_Studied', 'Attendance', 'Extracurricular_Activities',
       'Sleep_Hours', 'Previous_Scores', 'Internet_Access',
       'Tutoring_Sessions', 'Physical_Activity', 'Learning_Disabilities',
       'Exam_Score', 'Parental_Involvement_High', 'Parental_Involvement_Low',
       'Parental_Involvement_Medium', 'Access_to_Resources_High',
       'Access_to_Resources_Low', 'Access_to_Resources_Medium',
       'Motivation_Level_High', 'Motivation_Level_Low',
       'Motivation_Level_Medium', 'Family_Income_High', 'Family_Income_Low',
       'Family_Income_Medium', 'Teacher_Quality_High', 'Teacher_Quality_Low',
       'Teacher_Quality_Medium', 'Peer_Influence_Negative',
       'Peer_Influence_Neutral', 'Peer_Influence_Positive',
       'School_Type_Private', 'School_Type_Public',
       'Parental_Education_Level_College',
       'Parental_Education_Level_High School',
       'Parental_Education_Level_Postgraduate', 'Distance_from_Home_Far',
       'Distance_from_Home_Moderate', 'Dist

C:\Users\54911\AppData\Local\Temp\ipykernel_19844\1052431561.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset = dataset.replace({True: 1, False: 0})


,Hours_Studied,Attendance,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Internet_Access,Tutoring_Sessions,Physical_Activity,Learning_Disabilities,Exam_Score,...,School_Type_Private,School_Type_Public,Parental_Education_Level_College,Parental_Education_Level_High School,Parental_Education_Level_Postgraduate,Distance_from_Home_Far,Distance_from_Home_Moderate,Distance_from_Home_Near,Gender_Female,Gender_Male
0,23,84,0,7,73,1,0,3,0,67,...,0,1,0,1,0,0,0,1,0,1
1,19,64,0,8,59,1,2,4,0,61,...,0,1,1,0,0,0,1,0,1,0
2,24,98,1,7,91,1,2,4,0,74,...,0,1,0,0,1,0,0,1,0,1
3,29,89,1,8,98,1,1,4,0,71,...,0,1,0,1,0,0,1,0,0,1
4,19,92,1,6,65,1,3,4,0,70,...,0,1,1,0,0,0,0,1,1,0


# Regresion Lineal

In [67]:
X = dataset.drop('Exam_Score', axis=1).astype(float)
y = dataset['Exam_Score'].astype(float)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)
model = sm.OLS(y_train, x_train).fit()
predictions = model.predict(x_test)
print('Mean Squared Error:', mean_squared_error(y_test, predictions))
print(model.summary())

Mean Squared Error: 3.0210902271002684
                            OLS Regression Results                            
Dep. Variable:             Exam_Score   R-squared:                       0.706
Model:                            OLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                     450.9
Date:                Sat, 14 Dec 2024   Prob (F-statistic):               0.00
Time:                        15:41:11   Log-Likelihood:                -11121.
No. Observations:                5102   AIC:                         2.230e+04
Df Residuals:                    5074   BIC:                         2.248e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [68]:
CrossValidation = cross_val_score(LinearRegression(), X, y, scoring='neg_mean_squared_error', cv=10)
print('Cross Validation:', CrossValidation)
print('Mean Cross Validation:', np.mean(CrossValidation))
print('R2 Score:', r2_score(y_test, predictions))
print('Adjusted R2 Score:', 1 - (1-model.rsquared)*(len(y)-1)/(len(y)-x_train.shape[1]-1))

Cross Validation: [-6.42890793 -5.71055142 -3.33060914 -5.45727682 -4.82997906 -4.19943419
 -6.08116085 -2.28675646 -0.15924571 -4.58936869]
Mean Cross Validation: -4.307329028841502
R2 Score: 0.7889477715623027
Adjusted R2 Score: 0.7041072358747383


# Lasso

In [74]:
def findBestAlpha(X, y, alphas):
    mse = []
    for alpha in alphas:
        model = Lasso(alpha=alpha)
        mse.append(- np.mean(cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=10)))
    return mse

In [75]:
findBestAlpha(X, y, [0.0001, 0.001, 0.01, 0.1, 1, 10])

[4.307309687513696,
 4.307190807566209,
 4.314953861702552,
 5.094724442492111,
 6.670343820997061,
 10.643385213207228]

In [ ]:
lasso = Lasso(alpha=0.1)
lasso.fit(x_train, y_train)
predictions = lasso.predict(x_test)
print('Mean Squared Error:', mean_squared_error(y_test, predictions))
print('R2 Score:', r2_score(y_test, predictions))
for columna in range(len(X.columns)):
    print(X.columns[columna], lasso.coef_[columna])   

Mean Squared Error: 3.7720398907830255
R2 Score: 0.7364867101404767
Hours_Studied 0.28642453359531583
Attendance 0.19678240311853973
Extracurricular_Activities 0.13988131066175274
Sleep_Hours -0.0
Previous_Scores 0.04925653460944842
Internet_Access 0.0
Tutoring_Sessions 0.4254860534230773
Physical_Activity 0.05391760425022133
Learning_Disabilities -0.0
Parental_Involvement_High 0.7028735637101567
Parental_Involvement_Low -0.3774928129198202
Parental_Involvement_Medium 0.0
Access_to_Resources_High 0.6685370455689598
Access_to_Resources_Low -0.49935500570704916
Access_to_Resources_Medium -0.0
Motivation_Level_High 0.05537897713563886
Motivation_Level_Low -0.16508582626476634
Motivation_Level_Medium 0.0
Family_Income_High 0.013394714464467392
Family_Income_Low -0.2925594275129332
Family_Income_Medium 0.0
Teacher_Quality_High 0.1446379659164185
Teacher_Quality_Low -0.0
Teacher_Quality_Medium -0.0
Peer_Influence_Negative -0.025289402884783624
Peer_Influence_Neutral -0.0
Peer_Influence_Posit